<a href="https://colab.research.google.com/github/demid5111/dialog-intent-dl/blob/master/ipynb/PredictOnColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
! tar -xzf "/content/drive/My Drive/Projects/RFFI2016/dataset.tar.gz" -C /content
! ls "/content/feature_and_vector_seq" | wc -l

13156


In [3]:
! git clone https://github.com/demid5111/dialog-intent-dl.git
! mv dialog-intent-dl dialog_intent_dl
! pip install -r dialog-intent-dl/requirements.txt
! pip install tensorflow-gpu==1.13.1
! pip install numpy==1.15.4
# restart runtime

Cloning into 'dialog-intent-dl'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 164 (delta 7), reused 7 (delta 3), pack-reused 148
Receiving objects: 100% (164/164), 165.47 KiB | 6.89 MiB/s, done.
Resolving deltas: 100% (79/79), done.
     |████████████████████████████████| 1.4MB 2.8MB/s 
     |████████████████████████████████| 4.5MB 41.6MB/s 
     |████████████████████████████████| 22.6MB 4.2MB/s 
     |████████████████████████████████| 1.6MB 37.1MB/s 
     |████████████████████████████████| 12.2MB 36.8MB/s 
     |████████████████████████████████| 11.8MB 36.3MB/s 
     |████████████████████████████████| 31.2MB 1.6MB/s 
     |████████████████████████████████| 143kB 51.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/d1/ab/40/3bceea46922767e42986aef7606a600538ca80de6062dc266c
  Stored in directory: /root/.cache/pip/wheels/44/c0/34/6f98693a554301bdb405f8d65d95bbcd3e50180cbfdd98a94e

     |████████████████████████████████| 345.2MB 58kB/s 
     |████████████████████████████████| 368kB 50.7MB/s 
ERROR: tensorflow 1.14.0rc1 has requirement tensorflow-estimator<1.15.0rc0,>=1.14.0rc0, but you'll have tensorflow-estimator 1.13.0 which is incompatible.
  Found existing installation: tensorflow-estimator 1.14.0
    Uninstalling tensorflow-estimator-1.14.0:
      Successfully uninstalled tensorflow-estimator-1.14.0
     |████████████████████████████████| 13.9MB 2.8MB/s 
ERROR: tensorflow 1.14.0rc1 has requirement tensorflow-estimator<1.15.0rc0,>=1.14.0rc0, but you'll have tensorflow-estimator 1.13.0 which is incompatible.
ERROR: plotnine 0.5.1 has requirement pandas>=0.23.4, but you'll have pandas 0.23.1 which is incompatible.
ERROR: mizani 0.5.4 has requirement pandas>=0.23.4, but you'll have pandas 0.23.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement pandas~=0.24.0, but you'll have pandas 0.23.1 which is incompatible.
ERROR: fbprophet 0.5 has requiremen

In [2]:
import os
import tensorflow as tf
import keras
from keras.layers import Dense, Input, GlobalMaxPooling1D, Flatten, TimeDistributed, Bidirectional
from keras.layers import Conv1D, MaxPooling1D, Embedding, LSTM, Dropout, Activation, Permute
from keras import regularizers
from keras.models import Model, Sequential
from keras.callbacks import TensorBoard
from keras.backend import permute_dimensions
from dialog_intent_dl.ipynb.keras_handler.sequence_generator import SequenceGenerator
print(tf.__version__)

class PredictIntent():
    intents = {"": 0, " ": 0, "а": 1, "a": 1, "б": 2, "в": 3, "г": 4, "д": 5,
               "е": 6, "e": 6, "ж": 7, "з": 8, "3": 8, "и": 9, "к": 10,
               "л": 11, "м": 12, "н": 13, "о": 14, "п": 15,
               "р": 16, "с": 17, "т": 18, "у": 19, "ф": 20,
               "х": 21, "ц": 22, "ч": 23, "ш": 24, "щ": 25}
    general_intents = {"": 0, " ": 0, "а": 1, "a": 1, "б": 1, "в": 1, "г": 1, "д": 1,  # Информативно-воспроизводящий
                       "е": 2, "e": 2, "ж": 2, "з": 2, "3": 2, "и": 2, "к": 2,  # Эмотивно-консолидирующий
                       "л": 3, "м": 3, "н": 3, "о": 3, "п": 3,  # Манипулятивный тип, доминирование
                       "р": 4, "с": 4, "т": 4, "у": 4, "ф": 4,  # Волюнтивно-директивный
                       "х": 5, "ц": 5, "ч": 5, "ш": 5, "щ": 5}  # Контрольно-реактивный
    batch_size = 13
    max_sequence_length = 5
    intent_embedding_dim = 10
    num_units = 30
    validation_split = 0.2
    random_state = 42
    tb = TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False,
                     write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)

    def __init__(self, is_general=False, data_path = "../feature_and_vector_seq"):
        self.data_path = data_path
        if is_general:
            self.intent_index = self.general_intents
        else:
            self.intent_index = self.intents
        self.num_intents = max(self.intent_index.values()) + 1
        self.sg = SequenceGenerator(self.data_path, self.intent_index, self.max_sequence_length, self.validation_split,
                                    only_last=False,
                                    random_state=self.random_state)
        print('num_intents', self.num_intents)

    def build_CNN_model(self):
        embedding_layer = Embedding(self.num_intents,
                                    self.intent_embedding_dim,
                                    input_length=self.max_sequence_length - 1,
                                    trainable=True)

        sequence_input = Input(shape=(self.max_sequence_length - 1,), dtype='int32')
        embedded_sequences = embedding_layer(sequence_input)
        #         x_p = permute_dimensions(embedded_sequences, pattern=[0, 2, 1])

        x = Conv1D(128, 2, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(embedded_sequences)
        x = MaxPooling1D(1)(x)
        x = Dropout(0.2)(x)
        x = Conv1D(128, 3, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
        x = GlobalMaxPooling1D()(x)
        x = Dropout(0.2)(x)
        x = Dense(128, activation='relu')(x)
        preds = Dense(self.num_intents, activation='softmax')(x)

        model = Model(sequence_input, preds)
        model.compile(loss='categorical_crossentropy',
                      optimizer='rmsprop',
                      metrics=['acc'])
        self.model = model
        self.sg.only_last = True
        return self.model

    def build_MLP_model(self):
        embedding_layer = Embedding(self.num_intents,
                                    self.intent_embedding_dim,
                                    input_length=self.max_sequence_length - 1,
                                    trainable=True)

        model = Sequential()
        model.add(embedding_layer)
        model.add(Flatten())
        model.add(Dense(128, activation='relu', name="Dense1"
                        #                         , activity_regularizer=regularizers.l1(0.009)
                        , kernel_regularizer=regularizers.l2(0.0001)
                        #                         , bias_regularizer = regularizers.l2(0.009)
                        ))  #
        #         model.add(Dropout(0.2))
        model.add(Dense(self.num_intents, activation='softmax', name="Dense2"
                        #                         , activity_regularizer=regularizers.l1(0.009)
                        , kernel_regularizer=regularizers.l2(0.0001)
                        #                         , bias_regularizer = regularizers.l2(0.009)
                        ))
        #         model.add(Dropout(0.2))
        model.compile(loss='categorical_crossentropy',
                      optimizer='rmsprop',
                      metrics=['acc'])
        self.model = model
        self.sg.only_last = True
        return self.model

    def build_RNN_model(self):
        embedding_layer = Embedding(self.num_intents,
                                    self.intent_embedding_dim,
                                    input_length=self.max_sequence_length - 1,
                                    trainable=True)

        sequence_input = Input(shape=(self.max_sequence_length - 1,), dtype='int32')
        embedded_sequences = embedding_layer(sequence_input)

        #         x_t = permute_dimensions(embedded_sequences, pattern=[0, 2, 1])
        #         print('embedded_sequences',embedded_sequences.shape)
        #         print('x_t',x_t.shape)

        x = LSTM(self.num_units, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)(embedded_sequences)
        x = LSTM(self.num_units, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)(x)
        preds = TimeDistributed(Dense(self.num_intents, activation='softmax'))(x)
        #         preds = Dense(self.num_intents, activation='softmax')(x)
        #         print('preds.shape',preds.shape)

        model = Model(sequence_input, preds)
        model.compile(loss='categorical_crossentropy',
                      optimizer='rmsprop',
                      metrics=['acc'])
        self.model = model
        self.sg.only_last = False
        return self.model

    def build_BiRNN_model(self):
        embedding_layer = Embedding(self.num_intents,
                                    self.intent_embedding_dim,
                                    input_length=self.max_sequence_length - 1,
                                    trainable=True)
        model = Sequential()
        model.add(embedding_layer)
        #         model.add(Permute([1, 2]))
        model.add(Bidirectional(LSTM(self.num_units, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)))
        model.add(Bidirectional(LSTM(self.num_units, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)))
        #         model.add(Bidirectional(LSTM(self.num_units, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)))
        model.add(
            TimeDistributed(Dense(self.num_intents, activation='softmax', name="Dense2"), name="TimeDistributed2"))
        model.compile(loss='categorical_crossentropy',
                      optimizer='rmsprop',
                      metrics=['acc'])
        self.model = model
        return self.model

    def RNN_attention():

        embedding_layer = Embedding(self.num_intents,
                                    self.intent_embedding_dim,
                                    input_length=self.max_sequence_length - 1,
                                    trainable=True)
        model1 = Sequential()
        model1.add(embedding_layer)
        model1.add(LSTM(self.num_units, return_sequences=True))

        model2 = Sequential()
        model2.add(Dense(input_dim=input_dim, output_dim=step))
        model2.add(Activation('softmax'))  # Learn a probability distribution over each  step.
        # Reshape to match LSTM's output shape, so that we can do element-wise multiplication.
        model2.add(RepeatVector(self.num_units))
        model2.add(Permute(2, 1))

        model = Sequential()
        model.add(
            Merge([model1, model2], 'mul'))  # Multiply each element with corresponding weight a[i][j][k] * b[i][j]
        model.add(TimeDistributedMerge('sum'))  # Sum the weighted elements.

        attention = Dense(1, activation='tanh')(activations)
        attention = Flatten()(attention)
        attention = Activation('softmax')(attention)
        attention = RepeatVector(units)(attention)
        attention = Permute([2, 1])(attention)

        sent_representation = merge([activations, attention], mode='mul')
        model.compile(loss='categorical_crossentropy',
                      optimizer='rmsprop',
                      metrics=['acc'])
        self.model = model
        self.sg = SequenceGenerator(self.data_path, self.intent_index, self.max_sequence_length, self.validation_split,
                                    only_last=False)
        return self.model

    def fit_generator(self, epochs):
        history_nn = self.model.fit_generator(
            generator=self.sg.generate_batch(self.batch_size, subset='training'),
            steps_per_epoch=len(os.listdir(self.data_path)) * (1 - self.validation_split) // self.batch_size,
            epochs=epochs,
            validation_data=self.sg.generate_batch(self.batch_size, subset='validation'),
            validation_steps=len(os.listdir(self.data_path)) * self.validation_split // self.batch_size + 1,
            callbacks=[self.tb]
        )
        return history_nn
      
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.
    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = tf.graph_util.convert_variables_to_constants(
            session, input_graph_def, output_names, freeze_var_names)
        return frozen_graph
    
def save_model(session, model, name):
    frozen_graph = freeze_session(session, output_names=[out.op.name for out in model.outputs])
    tf.train.write_graph(frozen_graph, "../models", "{}".format(name), as_text=False)

Using TensorFlow backend.


1.13.1


In [3]:
PATH_TO_DATASET = "/content/feature_and_vector_seq"
pi = PredictIntent(data_path = PATH_TO_DATASET, is_general = False)
pi.batch_size = 7
pi.max_sequence_length = 5
pi.intent_embedding_dim = 10
pi.num_units = 30
pi.validation_split = 0.2
pi.random_state = 42

num_intents 26


In [4]:
pi.build_BiRNN_model()
history_BiRNN = pi.fit_generator(epochs = 1)
from keras import backend as K
save_model(K.get_session(), history_BiRNN, 'history_BiRNN.pb')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
1503/1503 [==============================] - 116s 77ms/step - loss: 2.3728 - acc: 0.2853 - val_loss: 2.0103 - val_acc: 0.3912


AttributeError: ignored

In [28]:
import pandas as pd
import re, json

def load_Doc2Vec_value(str1):
  pattern1 = re.compile('\n')
  pattern2 = re.compile(' ')
  str2 = pattern1.sub('', str1)
  str3 = pattern2.sub(', ', str2)
  return json.loads(str3)

file_and_path = "/content/feature_and_vector_seq/1-comm_inosmi_111_192344_output_10.h5"
df1 = pd.read_hdf(file_and_path, engine="python", encoding='cp1251')
df1["Doc2Vec"] = df1["Doc2Vec value"].apply(load_Doc2Vec_value)
df1


,ID of comment,ID of post,Likes,Intent analysis,Content analysis,Distance to parent,Distance to post,Doc2Vec value,Doc2Vec
0,192344,192344,0,,,0,1.1102230246251565e-16,[0.001903 0.008161 0.003022 0.021471 0.001911 ...,"[0.001903, 0.008161, 0.003022, 0.021471, 0.001..."
1,192366,192344,0,м,1,0.9047975317843413,0.9047975317843413,[0.015871 0.063059 0.027442 0.005918 0.067336 ...,"[0.015871, 0.063059, 0.027442, 0.005918, 0.067..."
2,192369,192344,1,а,3,0.9263160364648992,0.949031225401245,[-0.050479 -0.020054 0.010354 0.009482 -0.0072...,"[-0.050479, -0.020054, 0.010354, 0.009482, -0...."
3,192370,192344,0,ч,3,0.8536771566133271,1.0161022628113872,[-0.003400 0.004530 -0.011841 0.009115 0.01707...,"[-0.0034, 0.00453, -0.011841, 0.009115, 0.0170..."
4,192371,192344,1,н,4,0.4652375928796576,1.1818890054062847,[-0.017641 0.020937 -0.043087 0.017828 0.03717...,"[-0.017641, 0.020937, -0.043087, 0.017828, 0.0..."
5,192372,192344,0,р,4,0.5322816918635409,0.918104637002647,[0.014156 0.003831 0.001797 0.009888 0.004583 ...,"[0.014156, 0.003831, 0.001797, 0.009888, 0.004..."
6,192377,192344,0,ф,3,0.9230922955341927,0.5331857866510279,[-0.026679 -0.030724 0.026273 0.014249 0.00759...,"[-0.026679, -0.030724, 0.026273, 0.014249, 0.0..."
7,192381,192344,0,ч,4,0.8990398151377282,0.9101620933407938,[0.008495 0.029132 -0.002679 -0.007744 0.01133...,"[0.008495, 0.029132, -0.002679, -0.007744, 0.0..."
8,192395,192344,1,ч,3,1.0171192821636792,0.7188240607558654,[-0.009065 -0.010122 0.035569 -0.002556 -0.003...,"[-0.009065, -0.010122, 0.035569, -0.002556, -0..."


'[0.001903, 0.008161, 0.003022, 0.021471, 0.001911, 0.000769, 0.027428, -0.027831, -0.003073, -0.004980, 0.020644, -0.014960, 0.008604, 0.012825, -0.026068, 0.020403, 0.006612, 0.025695, 0.017430, -0.047408, -0.015395, -0.019598, 0.027972, -0.000601, 0.030375, 0.011772, 0.021578, -0.013088, -0.006355, -0.009098, -0.004229, 0.010379, 0.017166, -0.042873, 0.018200, -0.000433, -0.023537, 0.001672, -0.032163, 0.007928, 0.057112, 0.003129, 0.006764, 0.001141, -0.016615, -0.019479, -0.036450, 0.000785, -0.017258, -0.014166, 0.014459, -0.022591, -0.007363, -0.006674, -0.009149, -0.031017, 0.016442, 0.002262, -0.003950, 0.027914, 0.034671, 0.019261, -0.000262, 0.015857, -0.005982, 0.017371, -0.013526, -0.014391, 0.010246, 0.009417, -0.006874, -0.031878, -0.000649, 0.000668, 0.012658, 0.010711, 0.020093, -0.002385, 0.014615, -0.005328, 0.004366, -0.029982, 0.015387, -0.040169, -0.014782, -0.006836, -0.038126, -0.000182, 0.050316, -0.006597, -0.003478, -0.019885, -0.012636, -0.019870, -0.016595,

[0.001903,
 0.008161,
 0.003022,
 0.021471,
 0.001911,
 0.000769,
 0.027428,
 -0.027831,
 -0.003073,
 -0.00498,
 0.020644,
 -0.01496,
 0.008604,
 0.012825,
 -0.026068,
 0.020403,
 0.006612,
 0.025695,
 0.01743,
 -0.047408,
 -0.015395,
 -0.019598,
 0.027972,
 -0.000601,
 0.030375,
 0.011772,
 0.021578,
 -0.013088,
 -0.006355,
 -0.009098,
 -0.004229,
 0.010379,
 0.017166,
 -0.042873,
 0.0182,
 -0.000433,
 -0.023537,
 0.001672,
 -0.032163,
 0.007928,
 0.057112,
 0.003129,
 0.006764,
 0.001141,
 -0.016615,
 -0.019479,
 -0.03645,
 0.000785,
 -0.017258,
 -0.014166,
 0.014459,
 -0.022591,
 -0.007363,
 -0.006674,
 -0.009149,
 -0.031017,
 0.016442,
 0.002262,
 -0.00395,
 0.027914,
 0.034671,
 0.019261,
 -0.000262,
 0.015857,
 -0.005982,
 0.017371,
 -0.013526,
 -0.014391,
 0.010246,
 0.009417,
 -0.006874,
 -0.031878,
 -0.000649,
 0.000668,
 0.012658,
 0.010711,
 0.020093,
 -0.002385,
 0.014615,
 -0.005328,
 0.004366,
 -0.029982,
 0.015387,
 -0.040169,
 -0.014782,
 -0.006836,
 -0.038126,
 -0.0001